In [37]:
import sys
if ".." not in sys.path:
    sys.path.append("..")
import openpyxl
from energydata import energinet

import importlib
energinet = importlib.reload(energinet)

In [40]:
descriptor = {"resource_id": "connectionpointsingrid"}
service = energinet.EnergiDataServiceDk(descriptor)

In [41]:
list(energinet.fetch_edh(descriptor, 1))

[]

In [42]:
list(service.fetch_data())

[{'_id': 1001,
  'ConnectionPointCode': 'ØLS',
  'ConnectionPointName': 'Ølstykkegård',
  'MunicipalityNo': '240',
  'Municipality': 'Egedal',
  'RegionName': 'Hovedstaden',
  'ConnectionPointLatitude': 55.77148,
  'ConnectionPointLongitude': 12.143712,
  'NetCompanyName': 'REGIONALE NET',
  'region_code': 'dk_mun_240'},
 {'_id': 20,
  'ConnectionPointCode': 'AMK',
  'ConnectionPointName': 'Amagerkbst',
  'MunicipalityNo': '101',
  'Municipality': 'København',
  'RegionName': 'Hovedstaden',
  'ConnectionPointLatitude': 55.641284,
  'ConnectionPointLongitude': 12.609259,
  'NetCompanyName': 'KE TRANSMISSION',
  'region_code': 'dk_mun_101'},
 {'_id': 22,
  'ConnectionPointCode': 'AMV',
  'ConnectionPointName': 'Amagerværket',
  'MunicipalityNo': '101',
  'Municipality': 'København',
  'RegionName': 'Hovedstaden',
  'ConnectionPointLatitude': 55.685241,
  'ConnectionPointLongitude': 12.624494,
  'NetCompanyName': 'KE TRANSMISSION',
  'region_code': 'dk_mun_101'},
 {'_id': 23,
  'Connectio

In [68]:
metadata = service.fetch_metadata()
data = service.fetch_data()

In [69]:
connection_points = {
    row["ConnectionPointCode"] : row for row in data
}

In [145]:
{k: v for k, v in connection_points.items() if "strup" in v["ConnectionPointName"].lower()}

{'ESK': {'_id': 171,
  'ConnectionPointCode': 'ESK',
  'ConnectionPointName': 'Eskilstrup',
  'MunicipalityNo': '376',
  'Municipality': 'Guldborgsund',
  'RegionName': 'Sjælland',
  'ConnectionPointLatitude': 54.849705,
  'ConnectionPointLongitude': 11.893968,
  'NetCompanyName': 'SEAS/NVE TRANSMISSION',
  'region_code': 'dk_mun_376'},
 'EST': {'_id': 173,
  'ConnectionPointCode': 'EST',
  'ConnectionPointName': 'Estrupvej',
  'MunicipalityNo': '561',
  'Municipality': 'Esbjerg',
  'RegionName': 'Syddanmark',
  'ConnectionPointLatitude': 55.462691,
  'ConnectionPointLongitude': 8.474662,
  'NetCompanyName': 'Vestjyske Net A/S',
  'region_code': 'dk_mun_561'},
 'FARV': {'_id': 180,
  'ConnectionPointCode': 'FARV',
  'ConnectionPointName': 'Farstrup',
  'MunicipalityNo': '480',
  'Municipality': 'Nordfyn',
  'RegionName': 'Syddanmark',
  'ConnectionPointLatitude': 55.46349,
  'ConnectionPointLongitude': 10.1906,
  'NetCompanyName': 'VORES ELNET A/S',
  'region_code': 'dk_mun_480'},
 'HU

In [135]:
ws = openpyxl.load_workbook("../Energinet Extra Codes.xlsx").active

[column_names, *rows] = list(ws.values)
extra_station_codes = {}
for row in rows:
    d = dict(zip(column_names, row))
    if d["Code"] is not None:
        extra_station_codes[d["Name"]] = d["Code"]

In [136]:
extra_station_codes

{'A2': 'BJS',
 'AMF_132_F': 'AMV',
 'HASØ132_F': 'HAS',
 'ISHN132_N-$': 'ISH',
 'MRP1': 'TEG',
 'VSAK132_S1': 'VAL',
 'BIX_150_S1': 'BIL',
 'FRX_150_S1': 'FRT',
 'I4': 'ABS',
 'KLT_150_S1': 'KLF',
 'VKHI012_SFIK': 'VKE',
 'term_KT51-HT3': 'TJE'}

In [57]:
import io

In [59]:
netdata_url = "https://energinet.dk/-/media/76C3D434A7E2499F94E7DB66D3A2DE21.xlsx?la=da&hash=638E165AA57F627409CB9394D56D53614F5D7336"

with urllib.request.urlopen(netdata_url) as fileobj:
    buf = io.BytesIO()
    buf.write(fileobj.read())

buf.seek(0)
wb = openpyxl.load_workbook(buf)

In [61]:
wb.sheetnames

['Bus',
 'Line',
 'Transformer2',
 'Transformer3',
 'Generator',
 'Load',
 'Shunt',
 'HVDC',
 'PFModelInfo',
 'Ark1',
 'Ark2',
 'Ark3']

In [64]:
import networkx as nx

In [74]:
def get_sheet_data(ws):
    [_1, _2, _3, header, *rows] = ws.values
    for row in rows:
        yield {k: v for k, v in zip(header, row)}

In [146]:
import re
def match_bus_to_code(d, codes, extra_codes):
    extensions = {
        "ENDKW": "V",
        "ENDKE": "Ø"
    }
    name = d["Bus Name"]
    if name in extra_codes and extra_codes[name] in codes:
        return extra_codes[name]
    name_regex = re.compile(r"([^0-9_]+)_?(\d*)_?(.*)")
    match = name_regex.match(name)
    if not match:
        return
    bus_code = match.groups()[0]
    if bus_code in codes:
        return bus_code
    if d["Location Name"] in extensions:
        area_code = bus_code + extensions[d["Location Name"]]
        if area_code in codes:
            return area_code

In [147]:
energy_grid = nx.Graph()

In [148]:
bus_labels = {}
for d in get_sheet_data(wb["Bus"]):
    node_label = ("Bus", d["Bus Name"])
    bus_labels[d["Bus Index"]] = node_label
    attributes = {k: v for k, v in d.items()}
    connection_code = match_bus_to_code(d, list(connection_points), extra_station_codes)
    if connection_code is not None:
        attributes.update(connection_points[connection_code])
    energy_grid.add_nodes_from([(node_label, attributes)])

In [149]:
for d in get_sheet_data(wb["Line"]):
    energy_grid.add_edges_from([(bus_labels[d["Node 1"]], bus_labels[d["Node 2"]], d)])

In [150]:
for d in get_sheet_data(wb["Transformer2"]):
    node_label = ("Transformer3", d["2-Transformer Name"])    
    energy_grid.add_nodes_from([(node_label, d)])
    energy_grid.add_edges_from([
        (bus_labels[d["High.V Bus Index"]], node_label), 
        (bus_labels[d["Low.V Bus Index"]], node_label)
    ])

In [151]:
for d in get_sheet_data(wb["Transformer3"]):
    node_label = ("Transformer3", d["3_Transformer Name"])
    energy_grid.add_nodes_from([(node_label, d)])
    edges = [
        (bus_labels[d["High.V Bus Index"]], node_label),
        (bus_labels[d["Mid.V Bus Index"]], node_label),        
        (bus_labels[d["Low.V Bus Index"]], node_label)
    ]
    energy_grid.add_edges_from(edges)

In [152]:
for d in get_sheet_data(wb["Generator"]):
    node_label = ("Generator", d["Generator Name"])
    energy_grid.add_nodes_from([(node_label, d)])
    energy_grid.add_edge(node_label, bus_labels[d["Bus Index"]])

In [153]:
for d in get_sheet_data(wb["Load"]):
    node_label = ("Load", d["Load Name"])
    energy_grid.add_nodes_from([(node_label, d)])
    energy_grid.add_edge(node_label, bus_labels[d["Load Index"]])

In [154]:
for d in get_sheet_data(wb["Shunt"]):
    node_label = ("Shunt", d["Shunt Name"])
    energy_grid.add_nodes_from([(node_label, d)])
    energy_grid.add_edge(node_label, bus_labels[d["Bus Index"]])

In [155]:
for d in get_sheet_data(wb["HVDC"]):
    node_label = ("HVDC", d["HVDC Name"])
    energy_grid.add_nodes_from([(node_label, d)])
    energy_grid.add_edge(node_label, bus_labels[d["Bus Index"]])

In [156]:
[len(x) for x in nx.connected_components(energy_grid)]

[451, 561, 1, 1]

In [157]:
[x for x in nx.connected_components(energy_grid) if len(x) == 1]

[{('Bus', 'EDR_400_NIE1')}, {('Bus', 'EDR_400_NIE1(1)')}]

In [158]:
[x for x in energy_grid.nodes if "ConnectionPointLatitude" in energy_grid.nodes[x]]

[('Bus', 'A2'),
 ('Bus', 'ABS_150_S2-'),
 ('Bus', 'ADL_150_S1'),
 ('Bus', 'ALL_132_F1'),
 ('Bus', 'AMF_132_F'),
 ('Bus', 'AMK_132_A'),
 ('Bus', 'AMV_010_SFIK'),
 ('Bus', 'AMV_0173_SFIK'),
 ('Bus', 'AMV_132_A'),
 ('Bus', 'AND_150_S2'),
 ('Bus', 'ASR_150_S2'),
 ('Bus', 'ASR_400-ZL-'),
 ('Bus', 'ASR_400_S1'),
 ('Bus', 'ASV_0122_SFIK'),
 ('Bus', 'ASV_0215_SFIK'),
 ('Bus', 'ASV_132_F'),
 ('Bus', 'ASV_4001_SFIK'),
 ('Bus', 'ASV_400_K'),
 ('Bus', 'AVV_0122_SFIK'),
 ('Bus', 'AVV_0123_SFIK'),
 ('Bus', 'AVV_0171_SFIK'),
 ('Bus', 'AVV_0201_SFIK'),
 ('Bus', 'AVV_132_F'),
 ('Bus', 'AVV_132_SFIK'),
 ('Bus', 'AVV_400_SK.K'),
 ('Bus', 'BAG_132_F'),
 ('Bus', 'BBR_150_S1'),
 ('Bus', 'BDK_150_S1'),
 ('Bus', 'BDR_150_S1'),
 ('Bus', 'BED_150_N'),
 ('Bus', 'BJS_016'),
 ('Bus', 'BJS_132_SFIK'),
 ('Bus', 'BJS_220_SFIK'),
 ('Bus', 'BJS_400_K'),
 ('Bus', 'BLA_132_F'),
 ('Bus', 'BOR_132_F'),
 ('Bus', 'DYB_150_SA'),
 ('Bus', 'EBY'),
 ('Bus', 'EBY_132_F'),
 ('Bus', 'EDR_015_SFIK'),
 ('Bus', 'EDR_150_SV'),
 ('Bus',